In [0]:
%tensorflow_version 2.x
%load_ext tensorboard

In [0]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt

from absl import logging
print(tf.__version__)

In [0]:
def plot_series(time, series, format="-", start=0, end=None,color='blue'):
    plt.plot(time[start:end], series[start:end], format,color=color)
    plt.xlabel("Time")
    plt.ylabel("Value")
    plt.grid(True)

In [0]:
def trend(time, slope=0):
    return slope * time

time = np.arange(4 * 365 + 1, dtype="float32")
baseline = 10
slope = 0.03
series = trend(time, 0.1)  

plot_series(time,series)


In [0]:
def seasonal_pattern(season_time):
    """Just an arbitrary pattern, you can change it if you wish"""
    return np.where(season_time < 0.4,
                    np.cos(season_time * 2 * np.pi),
                    1 / np.exp(3 * season_time))

def seasonality(time, period, amplitude=1, phase=0):
    """Repeats the same pattern at each period"""
    season_time = ((time + phase) % period) / period
    return amplitude * seasonal_pattern(season_time)


baseline = 10
amplitude = 20

# Create the series
series = baseline + trend(time, slope) + seasonality(time, period=365, amplitude=amplitude)

plot_series(time,series)

In [0]:

def noise(time, noise_level=1, seed=None):
    rnd = np.random.RandomState(seed)
    return rnd.randn(len(time)) * noise_level

noise_level = 2


# Update with noise
series += noise(time, noise_level, seed=42)

plot_series(time, series)

In [0]:
split_time = 1000
time_train = time[:split_time]
x_train = series[:split_time]
time_test = time[split_time:]
x_test = series[split_time:]
train_nb = x_train.shape[0]
test_nb = x_test.shape[0]

In [0]:
def windowed_dataset(series, window_size, batch_size, shuffle_buffer):
  dataset = tf.data.Dataset.from_tensor_slices(series)
  dataset = dataset.window(window_size + 1, shift=1, drop_remainder=True)
  #print("window:",list(next(iter(dataset)).as_numpy_iterator()) )
  dataset = dataset.flat_map(lambda window: window.batch(window_size + 1))  
  #print("flat_map:",list(dataset.as_numpy_iterator()) )
  #dataset = dataset.map(lambda window: tf.expand_dims(window, axis=-1))
  #print("expanded map:",list(dataset.as_numpy_iterator()) )

  dataset = dataset.map(lambda window: (window[:-1], window[-1]))
  print("map:",list(dataset.as_numpy_iterator()) )

  dataset = dataset.shuffle(shuffle_buffer)
  dataset = dataset.batch(batch_size,drop_remainder=True)
  return dataset

tmp_x= np.array([1,3,2,4,6,2,1,1,2,1,6,7,2,4,6],dtype=np.float32 )
tmp_set = windowed_dataset(tmp_x, window_size=3, batch_size=4, shuffle_buffer=15)
#iter = iter(tmp_set)
#next(iter)#, next(iter) 
next(iter(tmp_set))

In [0]:
window_size = 20
batch_size = 32


train_ds = windowed_dataset(x_train, window_size, batch_size=128, shuffle_buffer= train_nb)
test_ds = windowed_dataset(x_test, window_size, batch_size=128, shuffle_buffer= test_nb)


RNN model

![Texte alternatif…](https://miro.medium.com/max/2944/1*obLEYSKh2nytc2Hu344psw.png)


In [0]:
tf.random.set_seed(1)
np.random.seed=1
def get_normal_tensor(shape):    
    return tf.random.normal(shape,mean=0.0, stddev=1.)*0.01

get_normal_tensor((2,5))

In [0]:
tf.random_normal_initializer(mean=0,stddev=1.0)((2,5))

In [0]:
class RNNCell(object):
    def __init__(self,n_x,n_a,w_init_fn):
        self.n_x = n_x
        self.n_a = n_a
        
        self.Wax = tf.Variable(w_init_fn((n_a,n_x)),dtype=tf.float32)
        self.Waa = tf.Variable(w_init_fn((n_a,n_a)),dtype=tf.float32)
        self.b =tf.Variable(tf.zeros((n_a,1)))
        

    def forward(self,a_prev,x):
        logging.info(f"cell fwd - x:{x}")
        logging.info(f"cell fwd - x:{x.shape}, a_prev: {a_prev.shape}")
        logging.info(f"cell fwd - wax:{self.Wax.shape}, waa: {self.Waa.shape}")
        z1 = tf.matmul(self.Wax, x,transpose_a=False,transpose_b=True)
        logging.info(f"cell fwd - z1:{z1.shape}")
        z2 = tf.matmul(self.Waa, a_prev)
        logging.info(f"cell fwd - z2:{z2.shape}")
        a_next = tf.tanh(z1+z2 + self.b) # hidden state
        logging.info(f"cell fwd - a_next:{a_next.shape}")
        return a_next

logging.set_verbosity(logging.INFO)
x_size = 1
cell = RNNCell(x_size,5,get_normal_tensor)


# a rnn takes an input of shape : nb_example, nb_timestep, nb_of_feature_per_timestep
# in this case we have : nb_example: 2, nb_timestep=3, nb_of_features_per_timestep = 1
x= np.array([[1,3,2],[2,1,1]],dtype=np.float32 )
logging.info(f"raw x: {x.shape}")
x=tf.expand_dims(x, axis=-1)
logging.info(f"reshaped x: {x.shape}")
# we call cell on first timestep
cell.forward(np.zeros((5,1),dtype=np.float32),x[:,0,:]).numpy()#.shape

In [0]:
class Model(object):
    def __init__(self,n_x, n_a, n_y,w_init=None):
        logging.info(f"model init - n_x:{n_x},n_a:{n_a},n_y:{n_y}")
        if w_init==None:
            w_init = get_normal_tensor
        self.cell =RNNCell(n_x,n_a,w_init)
        self.a0 = tf.zeros((n_a,1),dtype=tf.float32)
        self.Wya = tf.Variable(w_init((n_y,n_a)),dtype=tf.float32)
        self.by =tf.Variable(tf.zeros((n_y,1)))
        self.n_y = n_y


    def forward(self,X):
        logging.info(f"model fwd - X: {X.shape}")
        seq_len = X.shape[1]
        batch_size=X.shape[0]
        logging.info(f"model fwd - batch_size:{batch_size},seql_len: {seq_len}")
        a = {}
        a[-1] = tf.identity(self.a0)
        logging.info(f"model fwd - a[-1]:{a[-1].shape},{self.a0.shape}")
        

        #layer 1
        X=tf.expand_dims(X, axis=-1)
        logging.info(f"model fwd - X reshaped: {X.shape}")

        # recurrent layer
        for t in range(seq_len):
            a_next = self.cell.forward(a[t-1],X[:,t,:])
            a[t] = a_next
        # dense on last item
        logit = tf.matmul(self.Wya, a_next,transpose_a=False,transpose_b=False) + self.by
        y_hat = logit * 100.0
        return y_hat
        

logging.set_verbosity(logging.INFO)
model = Model(1,5,1)
x= np.array([[1,3,2],[2,1,1]],dtype=np.float32 )  
model.forward(x)

In [0]:
def compute_loss(y_pred,y_true):
    cce = tf.keras.losses.Huber()
    loss= cce(y_true, y_pred)
    return loss

compute_loss([2.1,2.3,4.2], [2.0,1.9,4.4])

In [0]:
def train_minibatch(model,X,y_true,learning_rate=5e-5, epoch=1):
    optimizer = tf.keras.optimizers.SGD(lr=learning_rate, momentum=0.9)
    with tf.GradientTape() as t:
        current_loss =  compute_loss(model.forward(X), y_true)

    tf.summary.scalar('loss', data=current_loss, step=epoch)
    dWya, dby, dWax,dWaa,db = t.gradient(current_loss, 
                                         [model.Wya, model.by,model.cell.Wax, model.cell.Waa,model.cell.b])
    
    w_g = zip([dWya, dby, dWax,dWaa,db], 
                                  [model.Wya, model.by,model.cell.Wax, model.cell.Waa,model.cell.b])
    optimizer.apply_gradients(w_g)

    return current_loss
logging.set_verbosity(logging.ERROR)
model = Model(1,5,1)
x= np.array([[1,3,2],[2,1,1]],dtype=np.float32 ) 
y_true = np.array([[4],[0]],dtype=np.float32 )   
model.forward(x)
train_minibatch(model,x,y_true,0.5,1)  

## Find the best learning rate

Try different learning rates on each epoch and identifies the one that has the lower loss.

In [0]:

model = Model(1,40,1)



def train(model,train_ds,nb_epochs=100,metric_func=tf.keras.metrics.MeanAbsoluteError(),lr=None, lr_cb=None):
    losses =[]
    metrics = []
    lrs = []
    epochs = range(nb_epochs)
    
    for epoch in epochs:
        batch_nb = 0

        if not lr_cb==None:
            lr = lr_cb.schedule(epoch)
        lrs.append(lr)
        tf.summary.scalar('learning_rate', data=lr, step=epoch)
        for x, y in train_ds:
            current_loss = train_minibatch(model, x, y,learning_rate=lr, epoch=epoch)
            batch_nb= batch_nb+1

        losses.append(current_loss.numpy())
        X_train, y_train = next(iter(train_ds))
        y_pred_train = model.forward(X_train)
        metric = metric_func(y_train,y_pred_train)
        metrics.append(metric.numpy())
        if epoch % 10 == 0:
            print(f"epoch {epoch} >> lr: {lr}, loss:{current_loss.numpy()},'metric mae:{metric.numpy()}")

    return losses, metrics,lrs





In [0]:
import datetime 
import os
logdir = "logs/rnn-timeseries/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
file_writer = tf.summary.create_file_writer(logdir + "/metrics")
file_writer.set_as_default()
logging.set_verbosity(logging.ERROR)
lr_schedule = tf.keras.callbacks.LearningRateScheduler(lambda epoch:1e-8 * 10**(epoch / 20))
losses, metrics,lrs=train(model, train_ds,nb_epochs=100,lr_cb=lr_schedule)

learning rate of 1e-5 

In [0]:
plt.semilogx(lrs, losses)
plt.axis([1e-8, 1e-4, 0, 30])

Learn with lr of 5e-5

In [0]:
logdir = "logs/rnn-timeseries/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
file_writer = tf.summary.create_file_writer(logdir + "/metrics")
file_writer.set_as_default()

logging.set_verbosity(logging.ERROR)
losses, metrics,lrs=train(model, train_ds,nb_epochs=100,lr=1e-5)



In [0]:
forecast=[]
for time in range(len(series) - window_size):
    forecast.append(model.forward(series[time:time + window_size][np.newaxis]))

forecast = forecast[split_time-window_size:]
results = np.array(forecast)[:, 0, 0]


plt.figure(figsize=(10, 6))

plot_series(time_test, x_test)
plot_series(time_test, results,color='red')

## Keras version

In [0]:

model = tf.keras.Sequential([
    tf.keras.layers.Input(shape=(window_size)),
    tf.keras.layers.Lambda(lambda x: tf.expand_dims(x, axis=-1)),
    # 5 is a too small hidden state -> use 40.
    tf.keras.layers.SimpleRNN(10, return_sequences=True),
    tf.keras.layers.SimpleRNN(10),
    tf.keras.layers.Dense(1),
    tf.keras.layers.Lambda(lambda x: x*100.0)
    ])

optimizer = tf.keras.optimizers.SGD(lr=1e-5, momentum=0.9)
#optimizer = tf.keras.optimizers.Adam(learning_rate=1e-5)
model.compile(optimizer=optimizer,
              loss=tf.keras.losses.Huber(),
              metrics=['mae'])

#model.summary()

In [0]:
tf.keras.backend.clear_session()
log_dir="logs/keras-rnn-timeseries/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard_cb = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1, write_graph=True)

model.fit(train_ds, epochs=100, callbacks=[tensorboard_cb])

In [0]:
model.weights

In [0]:
%tensorboard --logdir logs/keras-rnn-timeseries

In [0]:
forecast=[]
for time in range(len(series) - window_size):
    forecast.append(model.predict(series[time:time + window_size][np.newaxis]))

forecast = forecast[split_time-window_size:]
results = np.array(forecast)[:, 0, 0]


plt.figure(figsize=(10, 6))

plot_series(time_test, x_test)
plot_series(time_test, results,color='red')


Trash

In [0]:
def windowed_dataset(series, window_size, batch_size, shuffle_buffer):
  series = np.expand_dims(series, axis=1)
  #print(series)
  dataset = tf.data.Dataset.from_tensor_slices(series)
  dataset = dataset.window(window_size + 1, shift=1, drop_remainder=True)
  #print("window:",list(next(iter(dataset)).as_numpy_iterator()) )
  dataset = dataset.flat_map(lambda window: window.batch(window_size + 1))
  #print("flat_map:",list(dataset.as_numpy_iterator()) )

  dataset = dataset.map(lambda window: (window[:-1], window[-1]))
  #print("map:",list(dataset.as_numpy_iterator()) )
  dataset = dataset.shuffle(shuffle_buffer)
  dataset = dataset.batch(batch_size,drop_remainder=True)
  return dataset

tmp_x= np.array([1,3,2,4,6,2,1,1,2,1,6,7,2,4,6],dtype=np.float32 )
tmp_set = windowed_dataset(tmp_x, window_size=3, batch_size=4, shuffle_buffer=15)
#iter = iter(tmp_set)
#next(iter)#, next(iter) 
next(iter(tmp_set))

In [0]:
window_size = 20
batch_size = 32


train_ds = windowed_dataset(x_train, window_size, batch_size=128, shuffle_buffer= train_nb)
test_ds = windowed_dataset(x_test, window_size, batch_size=128, shuffle_buffer= test_nb)